In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers torch evaluate scikit-learn pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires f

In [3]:
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}
from transformers import Blip2Processor, Blip2ForConditionalGeneration

CSV_PATH = "/kaggle/input/images-with-vqas/merged_image_data_vqa.csv"
df = pd.read_csv(CSV_PATH)

df["image_path"] = df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

df["vqa_response"] = df["vqa_response"].apply(ast.literal_eval)
df = df.explode("vqa_response").reset_index(drop=True)
df[["question", "answer"]] = pd.DataFrame(df["vqa_response"].tolist(), index=df.index)

exists = df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
df = df[exists]

df = df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(df)}")


2025-05-10 20:28:36.989176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746908917.193221      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746908917.250811      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Skipping 0 rows with missing images
Total examples: 58124


In [4]:
df.head(5)

,image_path,question,answer
0,/kaggle/input/images-with-vqas/final_dataset/f...,What is the main color of the shoe?,Brown
1,/kaggle/input/images-with-vqas/final_dataset/f...,What is the pattern on the shoe?,Snakeskin
2,/kaggle/input/images-with-vqas/final_dataset/f...,Does the shoe have a tassel?,Yes
3,/kaggle/input/images-with-vqas/final_dataset/f...,What color are the drawer slides?,White
4,/kaggle/input/images-with-vqas/final_dataset/f...,What appears to be the material of the slides?,Metal


In [5]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model     = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16,
    device_map="auto"
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval().to(device)




Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [10]:


# VERIFY MODEL TYPE IMMEDIATELY AFTER LOADING
print(f"Type of 'model' after loading: {type(model)}")
assert isinstance(model, Blip2ForConditionalGeneration), "Model is not a Blip2ForConditionalGeneration instance!"

preds, refs = [], []
count  = 0
with torch.no_grad():
    for idx, row in df.iterrows():
        try:
            img = Image.open(row.image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: Image not found at {row.image_path}, skipping row {idx}")
            preds.append("IMAGE_NOT_FOUND")
            refs.append(row.answer.strip() if pd.notna(row.answer) else "")
            continue
        except Exception as e:
            print(f"Error loading image {row.image_path}: {e}, skipping row {idx}")
            preds.append("IMAGE_ERROR")
            refs.append(row.answer.strip() if pd.notna(row.answer) else "")
            continue

        original_question = row.question
        prompt = f"Based on the image, answer the following question with a single word. Question: {original_question} Answer:"


        inputs = processor(images=img, text=prompt, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=10,   # Try values like 5, 10.
            # num_beams=1,         # Optional: for greedy decoding, sometimes more direct
            # do_sample=False      # Optional: turn off sampling
        )
        pred_full = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

        
        words = pred_full.split()
        if words:
            pred = words[0] # Take the first word
            pred = pred.rstrip('.,;:!?') # Remove common trailing punctuation
        else:
            pred = "" # Handle empty prediction

        print(f"\nIdx: {idx}")
        print(f"Original Question: {original_question}")
        print(f"Prompt Used: '{prompt}'")
        print(f"Full Prediction: '{pred_full}'")
        print(f"Processed Prediction (Single Word Attempt): '{pred}'")

        count +=1
        preds.append(pred)
        refs.append(row.answer.strip() if pd.notna(row.answer) else "")
        print("count: ", count)
        # if (count == 100): # Limiting to 100 for the example
        #     print("\nReached count limit (100). Breaking loop.")
        #     break

print("length of predictions", len(preds))

Type of 'model' after loading: <class 'transformers.models.blip_2.modeling_blip_2.Blip2ForConditionalGeneration'>

Idx: 0
Original Question: What is the main color of the shoe?
Prompt Used: 'Based on the image, answer the following question with a single word. Question: What is the main color of the shoe? Answer:'
Full Prediction: 'brown'
Processed Prediction (Single Word Attempt): 'brown'
count:  1

Idx: 1
Original Question: What is the pattern on the shoe?
Prompt Used: 'Based on the image, answer the following question with a single word. Question: What is the pattern on the shoe? Answer:'
Full Prediction: 'python'
Processed Prediction (Single Word Attempt): 'python'
count:  2

Idx: 2
Original Question: Does the shoe have a tassel?
Prompt Used: 'Based on the image, answer the following question with a single word. Question: Does the shoe have a tassel? Answer:'
Full Prediction: 'yes'
Processed Prediction (Single Word Attempt): 'yes'
count:  3

Idx: 3
Original Question: What color are

In [7]:
! pip install git+https://github.com/google-research/bleurt.git
!pip install scikit-learn evaluate bert-score rouge-score \
            sentence-transformers rapidfuzz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-jwbpyhym
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-jwbpyhym
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=a01fa010d2f4fd559dbb5995687041f0ec520ab8cfb5d90f57b4a5d1721c5ea3
  Stored in directory: /tmp/pip-ephem-wheel-cache-o38tqfwk/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.1 MB/s eta 0:00:0000:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=61acd9476d7a5e1cdbc91f18735f85e0976946c6285781eefe581348667343f9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [15]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
# from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  # Will be same as acc

# ──────────────────────────────────────────────────────────────────────────────
# 7. Identify and Print Mismatched Predictions
# ──────────────────────────────────────────────────────────────────────────────
print("\n--- Mismatched Predictions ---")
mismatch_count = 0
# We iterate up to the length of preds_l, which is derived from 'preds'
# This assumes 'preds', 'refs', and the relevant part of 'df' are aligned
for i in range(len(preds_l)):
    prediction_normalized = preds_l[i]
    reference_normalized = refs_l[i]

    if prediction_normalized != reference_normalized:
        mismatch_count += 1
        print(f"\nMismatch #{mismatch_count} (Original Index: {i})")

        # Try to get the original question from the DataFrame
        # This assumes 'df' is available and 'i' corresponds to the row index used during inference.
        # If you only processed a subset of df (e.g., first 5 rows), this will work for i < 5.
        try:
            # If your inference loop used df.iterrows(), 'i' corresponds to the iteration count,
            # which might be the same as df.iloc[i] if you iterated from the start of df.
            # If you used a different way to get 'row' in your inference, adjust access to df.
            question = df.iloc[i]['question'] # Assuming 'preds' came from iterating 'df' from the start
            print(f"  Question: \"{question}\"")
        except (NameError, KeyError, IndexError) as e:
            # NameError if df is not defined
            # KeyError if 'question' column doesn't exist
            # IndexError if i is out of bounds for df (e.g., if df is smaller than len(preds_l))
            print(f"  Question: (Could not retrieve - {type(e).__name__})")

        print(f"  Model Predicted: \"{preds[i]}\" (Normalized: \"{prediction_normalized}\")")
        print(f"  Correct Answer:  \"{refs[i]}\" (Normalized: \"{reference_normalized}\")")
        print("-" * 30)

    if mismatch_count == 15:
        break

if mismatch_count == 0 and len(preds_l) > 0:
    print("No mismatches found! All (normalized) predictions exactly matched their references.")
elif len(preds_l) == 0:
    print("No predictions to compare.")

# Now you can add the other metrics calculation code (ROUGE, BERTScore, etc.) here if you wish

Exact-match Accuracy: 0.500
Exact-match Precision: 1.000
Exact-match Recall:    0.500
Exact-match F1:        0.667


--- Mismatched Predictions ---

Mismatch #1 (Original Index: 1)
  Question: "What is the pattern on the shoe?"
  Model Predicted: "python" (Normalized: "python")
  Correct Answer:  "Snakeskin" (Normalized: "snakeskin")
------------------------------

Mismatch #2 (Original Index: 4)
  Question: "What appears to be the material of the slides?"
  Model Predicted: "white" (Normalized: "white")
  Correct Answer:  "Metal" (Normalized: "metal")
------------------------------

Mismatch #3 (Original Index: 5)
  Question: "What shape are the screws?"
  Model Predicted: "screw" (Normalized: "screw")
  Correct Answer:  "Phillips" (Normalized: "phillips")
------------------------------

Mismatch #4 (Original Index: 7)
  Question: "What is the shape of the spool?"
  Model Predicted: "spool" (Normalized: "spool")
  Correct Answer:  "Round" (Normalized: "round")
------------------------

In [12]:
# --- ROUGE ---
print("\n--- ROUGE Scores ---")
rouge_eval_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []
for pred, ref in zip(preds_l, refs_l):
    if not pred or not ref: # Handle empty strings if any
        rouge1_scores.append(0.0)
        rougeL_scores.append(0.0)
        continue
    scores = rouge_eval_scorer.score(ref, pred) # Target, Prediction
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

if rouge1_scores:
    print(f"Average ROUGE-1 F1: {np.mean(rouge1_scores):.3f}")
    print(f"Average ROUGE-L F1: {np.mean(rougeL_scores):.3f}")
else:
    print("No ROUGE scores to compute (empty predictions or references).")


# --- BERTScore ---
print("\n--- BERTScore ---")
try:
    bert_eval_scorer = BERTScorer(lang="en", rescale_with_baseline=True, device=device)
    filtered_preds_l = [p for p, r in zip(preds_l, refs_l) if p and r]
    filtered_refs_l = [r for p, r in zip(preds_l, refs_l) if p and r]

    if filtered_preds_l and filtered_refs_l:
        P, R, F1_bert = bert_eval_scorer.score(filtered_preds_l, filtered_refs_l)
        print(f"Average BERTScore Precision: {P.mean():.3f}")
        print(f"Average BERTScore Recall:    {R.mean():.3f}")
        print(f"Average BERTScore F1:        {F1_bert.mean():.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for BERTScore.")
except Exception as e:
    print(f"Could not compute BERTScore: {e}")


# --- Levenshtein Normalized Similarity ---
print("\n--- Levenshtein Normalized Similarity ---")
lev_similarities = []
for pred, ref in zip(preds_l, refs_l):
    if not pred and not ref: # both empty, perfect match
        similarity = 1.0
    elif not pred or not ref: # one empty, other not, zero similarity
        similarity = 0.0
    else:
        similarity = Levenshtein.normalized_similarity(pred, ref)
    lev_similarities.append(similarity)

if lev_similarities:
    print(f"Average Levenshtein Normalized Similarity: {np.mean(lev_similarities):.3f}")
else:
    print("No Levenshtein similarities to compute.")


# --- Sentence Transformer Cosine Similarity ---
print("\n--- Sentence-BERT Cosine Similarity ---")
try:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    valid_pairs_indices = [i for i, (p, r) in enumerate(zip(preds_l, refs_l)) if p and r]
    sbert_preds = [preds_l[i] for i in valid_pairs_indices]
    sbert_refs = [refs_l[i] for i in valid_pairs_indices]

    if sbert_preds and sbert_refs:
        embeddings_preds = sbert_model.encode(sbert_preds, convert_to_tensor=True)
        embeddings_refs = sbert_model.encode(sbert_refs, convert_to_tensor=True)

        cosine_scores_sbert = util.cos_sim(embeddings_preds, embeddings_refs)
        pair_similarities = [cosine_scores_sbert[i, i].item() for i in range(len(sbert_preds))]
        print(f"Average Sentence-BERT Cosine Similarity: {np.mean(pair_similarities):.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for Sentence-BERT similarity.")
except Exception as e:
    print(f"Could not compute Sentence-BERT similarity: {e}")



--- ROUGE Scores ---
Average ROUGE-1 F1: 0.510
Average ROUGE-L F1: 0.510

--- BERTScore ---


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore Precision: 0.871
Average BERTScore Recall:    0.848
Average BERTScore F1:        0.858

--- Levenshtein Normalized Similarity ---
Average Levenshtein Normalized Similarity: 0.557

--- Sentence-BERT Cosine Similarity ---


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Average Sentence-BERT Cosine Similarity: 0.749
